In [1]:
import os
import numpy as np
import pandas as pd
import calendar 

import datetime
from dateutil.relativedelta import relativedelta
import pickle
import matplotlib.pyplot as plt

import scipy.stats.distributions as dist
from datetime import date

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

plt.style.use('ggplot')
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
%matplotlib inline

### Load data

In [2]:
data_folder = "data/"

In [3]:
train_df = pd.read_csv(data_folder+"train.csv")
test_df = pd.read_csv(data_folder+"test.csv")

stores_df = pd.read_csv(data_folder+"stores.csv")
trans_df = pd.read_csv(data_folder+"transactions.csv")

oil_df = pd.read_csv(data_folder+"oil.csv")
holidays_df = pd.read_csv(data_folder+"holidays_events.csv")

sample_sub_df = pd.read_csv(data_folder+"sample_submission.csv")

In [4]:
train_df.head(10)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0
6,6,2013-01-01,1,CELEBRATION,0.0,0
7,7,2013-01-01,1,CLEANING,0.0,0
8,8,2013-01-01,1,DAIRY,0.0,0
9,9,2013-01-01,1,DELI,0.0,0


In [5]:
train_df.store_nbr.unique()

array([ 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,  4,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  5, 50, 51, 52, 53, 54,  6,
        7,  8,  9])

In [6]:
train_df.id.nunique()

3000888

In [7]:
test_df.date.value_counts()

2017-08-24    1782
2017-08-22    1782
2017-08-27    1782
2017-08-17    1782
2017-08-23    1782
2017-08-30    1782
2017-08-29    1782
2017-08-25    1782
2017-08-18    1782
2017-08-16    1782
2017-08-26    1782
2017-08-31    1782
2017-08-21    1782
2017-08-19    1782
2017-08-28    1782
2017-08-20    1782
Name: date, dtype: int64

In [8]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
trans_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [10]:
oil_df.tail()

,date,dcoilwtico
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96
1217,2017-08-31,47.26


In [11]:
holidays_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [12]:
sample_sub_df

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


### Merge total dataset with store info

In [13]:
train_df_mrgstor = train_df.merge(stores_df,on="store_nbr",how="left")

In [14]:
train_df_mrgstor.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13


In [15]:
train_df_mrgstor = train_df_mrgstor.rename(columns={"type":"type_store"})

In [16]:
train_df_mrgstor.type_store.value_counts()

D    1000296
C     833580
A     500148
B     444576
E     222288
Name: type_store, dtype: int64

### Merge with holidays

In [17]:
holidays_df.locale.value_counts(dropna=False)

National    174
Local       152
Regional     24
Name: locale, dtype: int64

In [18]:
holidays_df_nat = holidays_df[holidays_df.locale=="National"]
holidays_df_loc = holidays_df[holidays_df.locale=="Local"]
holidays_df_reg = holidays_df[holidays_df.locale=="Regional"]

### Merge national first

In [19]:
train_df_mrgstor_holnat = train_df_mrgstor.merge(holidays_df_nat,on="date")
train_df_mrgstor_holloc = train_df_mrgstor.merge(holidays_df_loc,right_on=["date","locale_name"],left_on=["date","city"])
train_df_mrgstor_holreg = train_df_mrgstor.merge(holidays_df_reg,right_on=["date","locale_name"],left_on=["date","state"])

In [20]:
train_df_mrgstor_holnat = train_df_mrgstor_holnat[["id","type","locale","locale_name","description","transferred"]]
train_df_mrgstor_holloc = train_df_mrgstor_holloc[["id","type","locale","locale_name","description","transferred"]]
train_df_mrgstor_holreg = train_df_mrgstor_holreg[["id","type","locale","locale_name","description","transferred"]]

In [21]:
hol_df = pd.concat([train_df_mrgstor_holnat,train_df_mrgstor_holloc,train_df_mrgstor_holreg],axis=0)

hol_df.sample(4)

In [22]:
train_df_mrgstor_hol = train_df_mrgstor.merge(hol_df,on="id",how="left")

In [23]:
train_df_mrgstor_hol = train_df_mrgstor_hol.drop_duplicates(subset="id")

In [24]:
train_df_mrgstor_hol = train_df_mrgstor_hol.rename(columns={"type":"type_hol","locale":"locale_hol","locale_name":"locale_name_hol","description":"description_hol",\
                                                           "transferred":"transferred_hol"})

In [25]:
train_df_mrgstor_hol.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_store,cluster,type_hol,locale_hol,locale_name_hol,description_hol,transferred_hol
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False


### Merge oil price

In [26]:
train_df_mrgstor_hol_oil = train_df_mrgstor_hol.merge(oil_df,on="date",how="left")

### Too many holidays: which ones to consider

In [27]:
train_df_mrgstor_hol_oil.groupby("description_hol").sales.mean()

description_hol
Batalla de Pichincha                               337.347448
Black Friday                                       363.360708
Cantonizacion de Cayambe                           445.602558
Cantonizacion de El Carmen                         214.438109
Cantonizacion de Guaranda                          178.015915
Cantonizacion de Latacunga                         192.490970
Cantonizacion de Libertad                          180.356318
Cantonizacion de Quevedo                           200.778636
Cantonizacion de Riobamba                          171.422417
Cantonizacion del Puyo                              56.501323
Carnaval                                           332.882573
Cyber Monday                                       436.220250
Dia de Difuntos                                    418.357248
Dia de la Madre                                    320.515659
Dia de la Madre-1                                  447.135546
Dia del Trabajo                                    481

### quick and dirty: consider first 3 characters of holiday and use that to categorize

In [28]:
train_df_mrgstor_hol_oil['description_hol'] = train_df_mrgstor_hol_oil['description_hol'].str[:3]

In [29]:
train_df_mrgstor_hol_oil.head(1)

,id,date,store_nbr,family,sales,onpromotion,city,state,type_store,cluster,type_hol,locale_hol,locale_name_hol,description_hol,transferred_hol,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Pri,False,NaN


In [30]:
sales_by_fam = pd.DataFrame(train_df_mrgstor_hol_oil.groupby("family")["sales"].mean().sort_values()).reset_index()

In [31]:
sales_by_fam["fam_type"] = "REST"

In [32]:
sales_by_fam["fam_type"] = np.where((sales_by_fam["sales"]<30)&(sales_by_fam["family"]!="CELEBRATION"),sales_by_fam["fam_type"],sales_by_fam["family"])

In [33]:
sales_by_fam.fam_type.nunique()

17

In [34]:
sales_by_fam.family.nunique()

33

### Replace family type

In [35]:
train_df_mrgstor_hol_oil = train_df_mrgstor_hol_oil.merge(sales_by_fam[["family","fam_type"]],on="family")

In [36]:
len(train_df_mrgstor_hol_oil)

3000888

In [37]:
train_df_mrgstor_hol_oil.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_store,cluster,type_hol,locale_hol,locale_name_hol,description_hol,transferred_hol,dcoilwtico,fam_type
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Pri,False,NaN,REST
1,33,2013-01-01,10,AUTOMOTIVE,0.0,0,Quito,Pichincha,C,15,Holiday,National,Ecuador,Pri,False,NaN,REST
2,66,2013-01-01,11,AUTOMOTIVE,0.0,0,Cayambe,Pichincha,B,6,Holiday,National,Ecuador,Pri,False,NaN,REST
3,99,2013-01-01,12,AUTOMOTIVE,0.0,0,Latacunga,Cotopaxi,C,15,Holiday,National,Ecuador,Pri,False,NaN,REST
4,132,2013-01-01,13,AUTOMOTIVE,0.0,0,Latacunga,Cotopaxi,C,15,Holiday,National,Ecuador,Pri,False,NaN,REST


In [38]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


### Do analysis for 1 store 

In [39]:
train_df_no1 = train_df_mrgstor_hol_oil[train_df_mrgstor_hol_oil.store_nbr==1]

In [40]:
train_df_no1 = train_df_no1.drop(columns=["city","state","type_store","cluster","locale_name_hol","store_nbr"])

In [41]:
train_df_no1.head()

,id,date,family,sales,onpromotion,type_hol,locale_hol,description_hol,transferred_hol,dcoilwtico,fam_type
0,0,2013-01-01,AUTOMOTIVE,0.0,0,Holiday,National,Pri,False,NaN,REST
54,1782,2013-01-02,AUTOMOTIVE,2.0,0,NaN,NaN,NaN,NaN,93.14,REST
108,3564,2013-01-03,AUTOMOTIVE,3.0,0,NaN,NaN,NaN,NaN,92.97,REST
162,5346,2013-01-04,AUTOMOTIVE,3.0,0,NaN,NaN,NaN,NaN,93.12,REST
216,7128,2013-01-05,AUTOMOTIVE,5.0,0,Work Day,National,Rec,False,NaN,REST


In [44]:
train_df.iloc[1782]

id                   1782
date           2013-01-02
store_nbr               1
family         AUTOMOTIVE
sales                   2
onpromotion             0
Name: 1782, dtype: object

In [51]:
train_df_no1["date"] = pd.to_datetime(train_df_no1["date"])
train_df_no1["day_of_week"] = train_df_no1["date"].dt.dayofweek
train_df_no1["year"] = train_df_no1["date"].dt.year
train_df_no1["month"] = train_df_no1["date"].dt.month
train_df_no1["day_of_month"] = train_df_no1["date"].dt.day

In [52]:
train_df_no1 = train_df_no1.sort_values(by=["date","family"])

In [53]:
train_df_no1 = train_df_no1.drop(columns=["family"])

In [54]:
train_df_no1.day_of_week.value_counts()

1    7986
5    7953
0    7953
6    7920
4    7920
3    7920
2    7920
Name: day_of_week, dtype: int64

In [55]:
train_df_no1["weekend"] = np.where(train_df_no1["day_of_week"].isin([5,6]),1,0)

In [59]:
train_df_no1.head()

,date,sales,onpromotion,type_hol,locale_hol,description_hol,transferred_hol,dcoilwtico,fam_type,day_of_week,year,month,day_of_month,weekend
0,2013-01-01,0.0,0,Holiday,National,Pri,False,NaN,REST,1,2013,1,1,0
90936,2013-01-01,0.0,0,Holiday,National,Pri,False,NaN,REST,1,2013,1,1,0
181872,2013-01-01,0.0,0,Holiday,National,Pri,False,NaN,REST,1,2013,1,1,0
272808,2013-01-01,0.0,0,Holiday,National,Pri,False,NaN,BEVERAGES,1,2013,1,1,0
363744,2013-01-01,0.0,0,Holiday,National,Pri,False,NaN,REST,1,2013,1,1,0


### merge with store information 

In [42]:
len(train_df_no1)

55572

In [51]:
holidays_df.locale.value_counts()

National    174
Local       152
Regional     24
Name: locale, dtype: int64

In [53]:
holidays_df[holidays_df.locale=='National'].locale_name.value_counts()

Ecuador    174
Name: locale_name, dtype: int64

In [54]:
holidays_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [44]:
train_df_no1_mrg1 =  train_df_no1.merge(holidays_df,on="date",how="left")

In [47]:
train_df_no1_mrg1.sample(10)

,id,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,description,transferred
37392,1992279,2016-01-27,1,BEVERAGES,2131.000,2,NaN,NaN,NaN,NaN,NaN
47675,2528681,2016-11-23,1,MAGAZINES,9.000,0,NaN,NaN,NaN,NaN,NaN
42171,2240004,2016-06-14,1,PRODUCE,2108.374,0,NaN,NaN,NaN,NaN,NaN
21462,1142274,2014-10-05,1,GROCERY I,1070.000,2,NaN,NaN,NaN,NaN,NaN
11942,634421,2013-12-23,1,PREPARED FOODS,83.812,0,Additional,National,Ecuador,Navidad-2,False
1061,57029,2013-02-02,1,BREAD/BAKERY,302.402,0,NaN,NaN,NaN,NaN,NaN
6974,368885,2013-07-27,1,FROZEN FOODS,125.000,0,NaN,NaN,NaN,NaN,NaN
37121,1976267,2016-01-18,1,PREPARED FOODS,76.910,0,NaN,NaN,NaN,NaN,NaN
26178,1393533,2015-02-24,1,DELI,151.002,1,NaN,NaN,NaN,NaN,NaN
20672,1099508,2014-09-11,1,HARDWARE,1.000,0,NaN,NaN,NaN,NaN,NaN


In [41]:
len(train_df_no1_mrg1)

9306